In [15]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [16]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout,AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, GaussianNoise, GlobalMaxPooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [17]:
#Downloading dataset
!kaggle datasets download -d vuppalaadithyasairam/bone-fracture-detection-using-xrays

bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)
bone-fracture-detection-using-xrays.zip: Skipping, found more recently modified local copy (use --force to force download)


In [18]:
import zipfile
zip_ref = zipfile.ZipFile('/content/bone-fracture-detection-using-xrays.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [19]:
#Loading the images
x_train= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

x_test= keras.utils.image_dataset_from_directory(
    directory = '/content/archive (6)/val',
    labels='inferred',
    label_mode = 'int',
    batch_size=100,
    image_size=(256,256)
)

Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.
Found 8863 files belonging to 2 classes.
Found 600 files belonging to 2 classes.


In [20]:
class_names = x_train.class_names
class_names

['fractured', 'not fractured']

['fractured', 'not fractured']

In [21]:
# Normalizing
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

x_train = x_train.map(process)
x_test = x_test.map(process)

In [22]:
input = (256, 256, 3)

In [23]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers
from tensorflow.keras import Model

In [24]:
# Creating the base model
base_model = InceptionV3(weights='imagenet', include_top=False)

for layer in base_model.layers:
    layer.trainable = False

In [25]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.25)(x)
predictions = layers.Dense(1, activation='sigmoid')(x)

In [26]:
model = Model(inputs=base_model.input, outputs=predictions)

In [27]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [29]:
history = model.fit(x_train,epochs=5, batch_size=100, validation_data=x_test)

Epoch 1/5
89/89 [==============================] - 30s 317ms/step - loss: 0.0196 - accuracy: 0.9966 - val_loss: 0.8665 - val_accuracy: 0.7417
Epoch 2/5
89/89 [==============================] - 33s 354ms/step - loss: 0.0134 - accuracy: 0.9991 - val_loss: 0.7943 - val_accuracy: 0.7667
Epoch 3/5
89/89 [==============================] - 30s 321ms/step - loss: 0.0102 - accuracy: 0.9991 - val_loss: 0.8247 - val_accuracy: 0.7700
Epoch 4/5
89/89 [==============================] - 30s 324ms/step - loss: 0.0101 - accuracy: 0.9985 - val_loss: 0.9258 - val_accuracy: 0.7417
Epoch 5/5
89/89 [==============================] - 31s 323ms/step - loss: 0.0075 - accuracy: 0.9997 - val_loss: 0.8785 - val_accuracy: 0.7567
